# HW_3 - Web services 

In [1]:
import pandas as pd
import requests

dests = pd.read_csv('C:/Users/YaelD/Desktop/dests.txt', header=None, names=['Target']) # Importing the Data

def get_dis_dur(destination):
    # Returns distance and travel duration from TLV 
    # In case of illegal location - return None
    destinations = destination
    origins = 'Tel Aviv'
    units = 'kilometers'
    api_key = ''
    url="https://maps.googleapis.com/maps/api/distancematrix/json?destinations=%s&origins=%s&units=%s&key=%s" % (destinations, origins, units, api_key)
    try:
        response_data = requests.get(url)
        if not response_data.status_code == 200: 
            print("HTTP error",response_data.status_code)
        else:
            try:
                response_Data = response_data.json()
                if not response_Data["status"] == "OK":
                    print('Error in Google API')
                else:
                    Distance_km = response_Data['rows'][0]['elements'][0]['distance']['text']
                    Duration = response_Data['rows'][0]['elements'][0]['duration']['text']
                    return Distance_km, Duration
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return None, None

dests[['Distance_km', 'Duration']] = dests['Target'].apply(lambda x: pd.Series(get_dis_dur(x))) 

def get_lat_lng(address_string):
    # Returns longitude and latitude of given location
    # In case of illegal location - return None
    address = address_string
    api_key = ''
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,api_key)
    try:
        response_data = requests.get(url)
        if not response_data.status_code == 200: 
            print("HTTP error",response_data.status_code)
        else:
            try:
                response_Data = response_data.json()
                if not response_Data["status"] == "OK":
                    print('Error in Google API')
                else:
                    Longitude = response_Data['results'][0]["geometry"]["location"]['lng']
                    Latitude = response_Data['results'][0]["geometry"]["location"]['lat']
                    return Longitude, Latitude
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    return None, None

dests[['Longitude', 'Latitude']] = dests['Target'].apply(lambda x: pd.Series(get_lat_lng(x)))


In [2]:
dests

,Target,Distance_km,Duration,Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 3 mins,28.978359,41.008238
1,Amsterdam,"4,533 km",2 days 0 hours,4.904139,52.367573
2,Valletta,"3,793 km",2 days 3 hours,14.514100,35.899237
3,Basel,"4,093 km",1 day 20 hours,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 mins,51.531040,25.285447


In [3]:
dests = dests.sort_values('Distance_km',ascending=False)
dests.head(3)

,Target,Distance_km,Duration,Longitude,Latitude
1,Amsterdam,"4,533 km",2 days 0 hours,4.904139,52.367573
3,Basel,"4,093 km",1 day 20 hours,7.588576,47.559599
2,Valletta,"3,793 km",2 days 3 hours,14.514100,35.899237
